In [88]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
pd.set_option('display.max_columns', 500)


In [89]:
pd.read_csv('x_train.csv')

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kiduruni,Kilimanjaro,3,5,Hai,Masama Magharibi,125,True,GeoData Consultants Ltd,Water Board,Losaa Kia water supply,True,1999,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Igumbilo,Iringa,11,4,Njombe,Ikondo,56,True,GeoData Consultants Ltd,VWC,Ikondo electrical water sch,True,1996,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,Rufiji,Madungulu,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,Rufiji,Mwinyi,Dodoma,1,4,Chamwino,Mvumi Makulu,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


In [90]:
#pd.read_csv('y_train.csv')

In [91]:
x_train = pd.read_csv('x_train.csv').drop(['id'],axis=1)
y_train = pd.read_csv('y_train.csv').drop(['id'],axis=1)

In [92]:
cats = ['region','wpt_name','basin','lga','ward','extraction_type','extraction_type_group','extraction_type_class','management','management_group','payment',
        'payment_type','water_quality','quality_group','quantity','quantity_group','source','source_type','source_class',
       'waterpoint_type','waterpoint_type_group']
drops = ['region_code','district_code','recorded_by']
#drop columns with NA for now cuz lazy and looking for baseline
na_drops = ['funder','installer','subvillage','public_meeting','scheme_management','scheme_name','permit']

#drops2 = ['wpt_name','num_private']
#wpt_name too granular
#num_private

In [60]:
#x_train['management_group'].value_counts()

In [61]:
#x_train['public_meeting'].isna().sum()

In [62]:
x_train.isna().sum()

amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity

In [93]:
x_train = x_train.drop(drops,axis=1)
x_train = x_train.drop(na_drops,axis=1)
#x_train['date_recorded'] = pd.to_datetime(x_train['date_recorded'])
#x_train['date_recorded'] = x_train['date_recorded'] - min(x_train['date_recorded'])
x_train = x_train.drop(['date_recorded'],axis=1)

In [87]:
x_train['subvillage'].value_counts()

Madukani     508
Shuleni      506
Majengo      502
Kati         373
Mtakuja      262
            ... 
Liwigi         1
Lugese A       1
Komgobe        1
Matare A       1
Chamchuzi      1
Name: subvillage, Length: 19287, dtype: int64

In [65]:
len(x_train['wpt_name'].unique())

37400

In [94]:
cat = CatBoostClassifier(cat_features=cats)
cat.fit(x_train,y_train).score(x_train,y_train)


Learning rate set to 0.097451
0:	learn: 1.0219030	total: 275ms	remaining: 4m 34s
1:	learn: 0.9607508	total: 413ms	remaining: 3m 26s
2:	learn: 0.9113938	total: 567ms	remaining: 3m 8s
3:	learn: 0.8697001	total: 737ms	remaining: 3m 3s
4:	learn: 0.8325599	total: 873ms	remaining: 2m 53s
5:	learn: 0.8014269	total: 1.01s	remaining: 2m 48s
6:	learn: 0.7741483	total: 1.17s	remaining: 2m 45s
7:	learn: 0.7519532	total: 1.31s	remaining: 2m 42s
8:	learn: 0.7312502	total: 1.46s	remaining: 2m 40s
9:	learn: 0.7139690	total: 1.61s	remaining: 2m 39s
10:	learn: 0.6980530	total: 1.78s	remaining: 2m 39s
11:	learn: 0.6830223	total: 1.91s	remaining: 2m 37s
12:	learn: 0.6706399	total: 2.05s	remaining: 2m 35s
13:	learn: 0.6603359	total: 2.19s	remaining: 2m 34s
14:	learn: 0.6500158	total: 2.35s	remaining: 2m 34s
15:	learn: 0.6406627	total: 2.5s	remaining: 2m 33s
16:	learn: 0.6323772	total: 2.64s	remaining: 2m 32s
17:	learn: 0.6247909	total: 2.79s	remaining: 2m 32s
18:	learn: 0.6186954	total: 2.94s	remaining: 2m

158:	learn: 0.5105264	total: 24.5s	remaining: 2m 9s
159:	learn: 0.5103162	total: 24.7s	remaining: 2m 9s
160:	learn: 0.5100255	total: 24.8s	remaining: 2m 9s
161:	learn: 0.5099676	total: 24.9s	remaining: 2m 8s
162:	learn: 0.5095336	total: 25.1s	remaining: 2m 8s
163:	learn: 0.5094461	total: 25.2s	remaining: 2m 8s
164:	learn: 0.5092847	total: 25.4s	remaining: 2m 8s
165:	learn: 0.5092270	total: 25.5s	remaining: 2m 8s
166:	learn: 0.5091140	total: 25.7s	remaining: 2m 8s
167:	learn: 0.5087096	total: 25.8s	remaining: 2m 7s
168:	learn: 0.5086174	total: 26s	remaining: 2m 7s
169:	learn: 0.5084242	total: 26.1s	remaining: 2m 7s
170:	learn: 0.5080323	total: 26.3s	remaining: 2m 7s
171:	learn: 0.5078440	total: 26.4s	remaining: 2m 7s
172:	learn: 0.5077507	total: 26.6s	remaining: 2m 6s
173:	learn: 0.5076581	total: 26.7s	remaining: 2m 6s
174:	learn: 0.5074618	total: 26.8s	remaining: 2m 6s
175:	learn: 0.5072243	total: 27s	remaining: 2m 6s
176:	learn: 0.5070547	total: 27.1s	remaining: 2m 6s
177:	learn: 0.50

316:	learn: 0.4909017	total: 49s	remaining: 1m 45s
317:	learn: 0.4906932	total: 49.2s	remaining: 1m 45s
318:	learn: 0.4905887	total: 49.3s	remaining: 1m 45s
319:	learn: 0.4905543	total: 49.5s	remaining: 1m 45s
320:	learn: 0.4904905	total: 49.6s	remaining: 1m 44s
321:	learn: 0.4904345	total: 49.8s	remaining: 1m 44s
322:	learn: 0.4903479	total: 49.9s	remaining: 1m 44s
323:	learn: 0.4903240	total: 50s	remaining: 1m 44s
324:	learn: 0.4902713	total: 50.1s	remaining: 1m 44s
325:	learn: 0.4901962	total: 50.2s	remaining: 1m 43s
326:	learn: 0.4901415	total: 50.4s	remaining: 1m 43s
327:	learn: 0.4900858	total: 50.5s	remaining: 1m 43s
328:	learn: 0.4900102	total: 50.7s	remaining: 1m 43s
329:	learn: 0.4898806	total: 50.8s	remaining: 1m 43s
330:	learn: 0.4896378	total: 51s	remaining: 1m 43s
331:	learn: 0.4895212	total: 51.2s	remaining: 1m 42s
332:	learn: 0.4894723	total: 51.3s	remaining: 1m 42s
333:	learn: 0.4894509	total: 51.5s	remaining: 1m 42s
334:	learn: 0.4894256	total: 51.6s	remaining: 1m 42s

471:	learn: 0.4790706	total: 1m 12s	remaining: 1m 21s
472:	learn: 0.4789465	total: 1m 12s	remaining: 1m 21s
473:	learn: 0.4789131	total: 1m 13s	remaining: 1m 21s
474:	learn: 0.4787925	total: 1m 13s	remaining: 1m 20s
475:	learn: 0.4787422	total: 1m 13s	remaining: 1m 20s
476:	learn: 0.4786093	total: 1m 13s	remaining: 1m 20s
477:	learn: 0.4785386	total: 1m 13s	remaining: 1m 20s
478:	learn: 0.4785151	total: 1m 13s	remaining: 1m 20s
479:	learn: 0.4784721	total: 1m 13s	remaining: 1m 20s
480:	learn: 0.4783711	total: 1m 14s	remaining: 1m 19s
481:	learn: 0.4783331	total: 1m 14s	remaining: 1m 19s
482:	learn: 0.4782802	total: 1m 14s	remaining: 1m 19s
483:	learn: 0.4782222	total: 1m 14s	remaining: 1m 19s
484:	learn: 0.4781142	total: 1m 14s	remaining: 1m 19s
485:	learn: 0.4779710	total: 1m 14s	remaining: 1m 19s
486:	learn: 0.4779239	total: 1m 14s	remaining: 1m 18s
487:	learn: 0.4778141	total: 1m 15s	remaining: 1m 18s
488:	learn: 0.4777842	total: 1m 15s	remaining: 1m 18s
489:	learn: 0.4777202	total:

626:	learn: 0.4687447	total: 1m 36s	remaining: 57.3s
627:	learn: 0.4686782	total: 1m 36s	remaining: 57.1s
628:	learn: 0.4686240	total: 1m 36s	remaining: 57s
629:	learn: 0.4685929	total: 1m 36s	remaining: 56.8s
630:	learn: 0.4685259	total: 1m 36s	remaining: 56.7s
631:	learn: 0.4684934	total: 1m 37s	remaining: 56.5s
632:	learn: 0.4684469	total: 1m 37s	remaining: 56.3s
633:	learn: 0.4684103	total: 1m 37s	remaining: 56.2s
634:	learn: 0.4683333	total: 1m 37s	remaining: 56s
635:	learn: 0.4683036	total: 1m 37s	remaining: 55.9s
636:	learn: 0.4681514	total: 1m 37s	remaining: 55.7s
637:	learn: 0.4681119	total: 1m 37s	remaining: 55.6s
638:	learn: 0.4681020	total: 1m 38s	remaining: 55.4s
639:	learn: 0.4680200	total: 1m 38s	remaining: 55.2s
640:	learn: 0.4679937	total: 1m 38s	remaining: 55.1s
641:	learn: 0.4679592	total: 1m 38s	remaining: 54.9s
642:	learn: 0.4679297	total: 1m 38s	remaining: 54.8s
643:	learn: 0.4677858	total: 1m 38s	remaining: 54.6s
644:	learn: 0.4677710	total: 1m 38s	remaining: 54.

782:	learn: 0.4592937	total: 1m 59s	remaining: 33.1s
783:	learn: 0.4591132	total: 1m 59s	remaining: 33s
784:	learn: 0.4590696	total: 1m 59s	remaining: 32.8s
785:	learn: 0.4590246	total: 2m	remaining: 32.7s
786:	learn: 0.4589892	total: 2m	remaining: 32.5s
787:	learn: 0.4589226	total: 2m	remaining: 32.4s
788:	learn: 0.4588611	total: 2m	remaining: 32.2s
789:	learn: 0.4587829	total: 2m	remaining: 32.1s
790:	learn: 0.4587020	total: 2m	remaining: 31.9s
791:	learn: 0.4586503	total: 2m	remaining: 31.8s
792:	learn: 0.4585588	total: 2m 1s	remaining: 31.6s
793:	learn: 0.4585217	total: 2m 1s	remaining: 31.4s
794:	learn: 0.4584852	total: 2m 1s	remaining: 31.3s
795:	learn: 0.4584466	total: 2m 1s	remaining: 31.1s
796:	learn: 0.4584265	total: 2m 1s	remaining: 31s
797:	learn: 0.4584063	total: 2m 1s	remaining: 30.8s
798:	learn: 0.4583711	total: 2m 1s	remaining: 30.7s
799:	learn: 0.4583035	total: 2m 2s	remaining: 30.5s
800:	learn: 0.4582261	total: 2m 2s	remaining: 30.4s
801:	learn: 0.4581680	total: 2m 2s

940:	learn: 0.4503127	total: 2m 23s	remaining: 8.98s
941:	learn: 0.4502420	total: 2m 23s	remaining: 8.83s
942:	learn: 0.4502063	total: 2m 23s	remaining: 8.68s
943:	learn: 0.4500869	total: 2m 23s	remaining: 8.53s
944:	learn: 0.4500143	total: 2m 23s	remaining: 8.37s
945:	learn: 0.4499685	total: 2m 24s	remaining: 8.22s
946:	learn: 0.4498940	total: 2m 24s	remaining: 8.07s
947:	learn: 0.4498509	total: 2m 24s	remaining: 7.92s
948:	learn: 0.4496942	total: 2m 24s	remaining: 7.76s
949:	learn: 0.4496321	total: 2m 24s	remaining: 7.62s
950:	learn: 0.4495955	total: 2m 24s	remaining: 7.46s
951:	learn: 0.4495780	total: 2m 24s	remaining: 7.31s
952:	learn: 0.4495241	total: 2m 25s	remaining: 7.16s
953:	learn: 0.4494565	total: 2m 25s	remaining: 7.01s
954:	learn: 0.4494071	total: 2m 25s	remaining: 6.85s
955:	learn: 0.4493797	total: 2m 25s	remaining: 6.7s
956:	learn: 0.4493408	total: 2m 25s	remaining: 6.55s
957:	learn: 0.4493200	total: 2m 25s	remaining: 6.4s
958:	learn: 0.4492283	total: 2m 26s	remaining: 6

0.859006734006734

In [54]:
x_train

,amount_tsh,gps_height,longitude,latitude,wpt_name,num_private,basin,region,lga,ward,population,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,6000.0,1390,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,Ludewa,Mundindi,109,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,0.0,1399,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,Serengeti,Natta,280,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,25.0,686,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,Simanjiro,Ngorika,250,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,0.0,263,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,Nanyumbu,Nanyumbu,58,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,0.0,0,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,Karagwe,Nyakasimbi,0,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,10.0,1210,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kilimanjaro,Hai,Masama Magharibi,125,1999,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,4700.0,1212,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Iringa,Njombe,Ikondo,56,1996,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,0.0,0,34.017087,-8.750434,Mashine,0,Rufiji,Mbeya,Mbarali,Chimala,0,0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,0.0,0,35.861315,-6.378573,Mshoro,0,Rufiji,Dodoma,Chamwino,Mvumi Makulu,0,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


In [95]:
x_train['water_quality'].value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [96]:
x_train['quality_group'].value_counts()

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

In [97]:
x_train['source'].value_counts()

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [98]:
x_train['source_type'].value_counts()

spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64

In [100]:
x_train['extraction_type'].value_counts()

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64

In [101]:
x_train['extraction_type_group'].value_counts()

gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
india mark iii        98
Name: extraction_type_group, dtype: int64

In [103]:
x_train['extraction_type_class'].value_counts()

gravity         26780
handpump        16456
other            6430
submersible      6179
motorpump        2987
rope pump         451
wind-powered      117
Name: extraction_type_class, dtype: int64

In [106]:
pd.read_csv('x_test.csv')

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,2011-02-24,Danida,34,Da,38.852669,-6.582841,Kwambwezi,0,Wami / Ruvu,Yombo,Pwani,6,1,Bagamoyo,Yombo,20,True,GeoData Consultants Ltd,VWC,Bagamoyo wate,True,1988,mono,mono,motorpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
14846,18990,1000.0,2011-03-21,Hiap,0,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0,Pangani,Mkondoa,Tanga,4,7,Kilindi,Mvungwe,2960,True,GeoData Consultants Ltd,VWC,NaN,False,1994,nira/tanira,nira/tanira,handpump,vwc,user-group,pay annually,annually,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
14847,28749,0.0,2013-03-04,NaN,1476,NaN,34.739804,-4.585587,Bwawani,0,Internal,Juhudi,Singida,13,2,Singida Rural,Ughandi,200,True,GeoData Consultants Ltd,VWC,NaN,NaN,2010,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,dam,dam,surface,communal standpipe,communal standpipe
14848,33492,0.0,2013-02-18,Germany,998,DWE,35.432732,-10.584159,Kwa John,0,Lake Nyasa,Namakinga B,Ruvuma,10,2,Songea Rural,Maposeni,150,True,GeoData Consultants Ltd,VWC,Mradi wa maji wa maposeni,True,2009,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe


In [107]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    return summary
resumetable(x_train)

Dataset Shape: (59400, 28)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,amount_tsh,float64,0,98,6000.0,0.0,25.0
1,gps_height,int64,0,2428,1390,1399,686
2,longitude,float64,0,57516,34.938093,34.698766,37.460664
3,latitude,float64,0,57517,-9.856322,-2.147466,-3.821329
4,wpt_name,object,0,37400,none,Zahanati,Kwa Mahundi
5,num_private,int64,0,65,0,0,0
6,basin,object,0,9,Lake Nyasa,Lake Victoria,Pangani
7,region,object,0,21,Iringa,Mara,Manyara
8,lga,object,0,125,Ludewa,Serengeti,Simanjiro
9,ward,object,0,2092,Mundindi,Natta,Ngorika
